In [ ]:
import jax
from jax import grad, jit
import jax.numpy as jnp

In [ ]:
@jit
def f(x):
  for i in range(3):
    x = 2 * x
  return x

print(f(3))

In [ ]:
@jit
def g(x):
  y = 0.
  for i in range(x.shape[0]):
    y = y + x[i]
  return y

print(g(jnp.array([1., 2., 3.])))

# jax.make_jaxpr(g)(jnp.array([1., 2., 3., 4.2]))
# 运行上面的代码可以看到，如果 x 的长度不一样，编译的结果也不一样
# 对于不同长度的输入，jax 会重新编译

In [ ]:
# static_argnames 的一个例子

def f(x):
  if x < 3:
    return 3. * x ** 2
  else:
    return -4 * x

f = jit(f, static_argnames='x')

print(f(2.))

In [ ]:
# static_argnames 的另一个例子

def f(x, n):
  y = 0.
  for i in range(n):
    y = y + x[i]
  return y

f = jit(f, static_argnames='n')

f(jnp.array([2., 3., 4.]), 2)

In [ ]:
# static_argnames 的更多例子
# static_argnames 可以指定多个参数为静态参数
# 或者可以使用 static_argnums 来指定参数的位置

def example_fun(length, val):
  return jnp.ones((length,)) * val
# un-jit'd works fine
print(example_fun(5, 4))

# 注意，这个函数 example_fun 不能够直接 jit，因为它的第一个参数是一个变量

# static_argnames tells JAX to recompile on changes at these argument positions:
good_example_jit = jit(example_fun, static_argnames='length')
# first compile
print(good_example_jit(10, 4))
# recompiles
print(good_example_jit(5, 8))

# 如果想指定更多的参数为静态参数，可以使用 static_argnames
more_example_jit = jit(example_fun, static_argnames=['length', 'val'])
# 或者换个写法
more_example_jit = jit(example_fun, static_argnums=(0, 1))
# first compile
print(more_example_jit(10, 4))
# recompiles
print(more_example_jit(5, 8))
# 这里的 length 和 val 都是静态参数
# 但是如果 length 和 val 都是动态参数，那么就会报错
print(more_example_jit(12, 8)) # 这里的 length 和 val 都是动态参数

In [ ]:
# 如果需要在 jit 版本的函数中使用 print 语句，可以使用 jax.debug.print

import jax

@jit
def f(x):
    jax.debug.print("The first printed number is {x}", x=x)
    y = 2 * x
    print(y)
    jax.debug.print("The second printed number is {}", y)
    return y
f(2)

# Structured control flow primitives

In [ ]:
# jax.lax.cond

from jax import lax

operand = jnp.array([0.])
print(lax.cond(True, lambda x: x+1, lambda x: x-1, operand))
print(lax.cond(False, lambda x: x+1, lambda x: x-1, operand))

In [ ]:
# jax.lax.select 和 jax.numpy.where

import jax
import jax.numpy as jnp
from jax import lax

# 条件：布尔数组
pred = jnp.array([True, False, True])

# 两个预先计算的数组
x = jnp.array([1, 2, 3])
y = jnp.array([10, 20, 30])

# 在 x 和 y 之间逐位选择
out = lax.select(pred, x, y)
print(out)  # [ 1 20  3]

out_jnp = jnp.where(pred, x, y)
print(out_jnp)  # [ 1 20  3]

In [ ]:
# jax.lax.switch

def branch_0(x): return x + 100
def branch_1(x): return x * 10
def branch_2(x): return x - 50

branches = [branch_0, branch_1, branch_2]
index = 2  # 动态 index 指示使用哪个分支
num = 5

result = lax.switch(index, branches, num)
print(result)


In [ ]:
# 使用 vmap 向量化 lax.switch

def demo(index, num):
    branches = [branch_0, branch_1, branch_2]
    return lax.switch(index, branches, num)

vectorized_demo = jax.vmap(demo, in_axes=(0, 0))

# 多个动态 index 和 num
indices = jnp.array([0, 1, 2])
nums = jnp.array([5, 7, 8])

result = vectorized_demo(indices, nums)
print(result)  # 输出: [105 70 -42]

In [ ]:
# jax.numpy.select 用于二维数组
# 不学习更高维了，因为不常用

conditions = [
    jnp.array([True, False, False]),
    jnp.array([False, True, True]),
    jnp.array([True, True, False]),
]

# conditions = jnp.array(conditions) 可以不需要

choices = [
    jnp.array([1, 2, 3]),
    jnp.array([4, 5, 6]),
    jnp.array([7, 8, 9]),
]

# choices = jnp.array(choices) 可以不需要

result = jnp.select(conditions, choices, default=-1)
print(result)

In [ ]:
# jax.numpy.piecewise

import jax.numpy as jnp

x = jnp.array([-3.0, 0.0, -1.5, 3.5])

conds = [
    x < 0,
    (x >= 0) & (x < 2),
    x >= 2
]

i = 0

def f1(x):
    global i
    i += 1
    y = jnp.full_like(x, -1.0)
    jax.debug.print("f1 is called: y is of type {}", type(y))
    jax.debug.print("f1 is called: y is of shape {}", y.shape) # 这个句子并不能正确返回y的形状 因为这些代码都是函数式的
    return y

f_list = [
    #-1, 这样写更简单
    # lambda x: -1, 这样写会报错
    f1,
    lambda x: x ** 2,
    lambda x: x + 10,
]

result = jnp.piecewise(x, conds, f_list)
print(result)  # [-1.     0.     2.25  13.5 ]
print(i) #会返回 1 因为纯函数式编程的特性

In [ ]:
# jax.lax.while_loop

init_val = 0
cond_fun = lambda x: x < 10
body_fun = lambda x: x+1
lax.while_loop(cond_fun, body_fun, init_val)
# --> array(10, dtype=int32)

In [ ]:
# jax.lax.fori_loop

init_val = 0
start = 0
stop = 10

#body_fun = lambda i,x: x+i
def body_fun(i, x):
    return x + i
lax.fori_loop(start, stop, body_fun, init_val)
# --> array(45, dtype=int32)

In [ ]:
# 逻辑运算符

def python_check_positive_even(x):
  is_even = x % 2 == 0
  # `and` short-circults, so when `is_even` is `False`, `x > 0` is not evaluated.
  return is_even and (x > 0)

@jit
def jax_check_positive_even(x):
  is_even = x % 2 == 0
  # `logical_and` does not short circuit, so `x > 0` is always evaluated.
  return jnp.logical_and(is_even, x > 0)

print(python_check_positive_even(24))
print(jax_check_positive_even(24))

x = jnp.array([-1, 2, 5])
print(jax_check_positive_even(x))

In [ ]:
# python_check_positive_even(x) 这个会报错 与上面的代码形成对比

# print(python_check_positive_even(x)) 这个会报错，下面来探究原因

print([True, True] and [False, True])

print([True, False] and True)

print((x%2 == 0).tolist() and (x > 0).tolist())

print(type(x%2 == 0)) # <class 'jax.interpreters.xla.DeviceArray'>
# x%2 == 0 and x > 0 # 这个会报错
# 报错 ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
# 这个错误是因为 x%2 == 0 和 x > 0 都是布尔 jnp.ndarray